# npy2parquet-module

`npy2parquet` is a module that can convert a pandas dataframe that lists numpy arrays into a parquet-file, where all of the numpy data is in one column.

One can use the module from the command line, when the input format is CSV.

In [1]:
!python -m npy2parquet --help 

usage: This program will write numpy arrays listed in a CSV into a single parquet file

positional arguments:
  csv                   CSV that contains the names of the npy files in one
                        column
  output                Output .parquet-file

optional arguments:
  -h, --help            show this help message and exit
  -n NPYFILE_COLUMN, --npyfile-column NPYFILE_COLUMN
                        Column that contains the names of the numpy arrays
                        (default: npyfile)
  -d DATA_COLUMN, --data-column DATA_COLUMN
                        Column that will store the numpy array contents
                        (default: data)
  -b BATCH_SIZE, --batch-size BATCH_SIZE
                        How many arrays should be stored in one parquet row-
                        group (default: 50)
  -s, --shuffle         Shuffle rows before storing the data(default: False)
  -r SEED, --seed SEED  Seed used for shuffling (default: 10)
  -f, --overwrite       Overwrite

## Requirements

This module requires `numpy`-, `pandas`- and `pyarrow`-modules.

You can install them with:

```sh
pip install numpy pandas pyarrow
```
for virtual environments or
```sh
conda install numpy pandas pyarrow
```
for conda environments.

## Create test numpy arrays

Let's create some 10000 test numpy arrays of size 100x100:

In [2]:
%run create_test_arrays.py -n 10000 -x 100 -y 100

Creating 10000 arrays of shape (100, 100)
         n    param1    param2                      npyfile
0        0  0.215664  0.185119     numpy_arrays/array-0.npy
1        1  0.038707  0.180713     numpy_arrays/array-1.npy
2        2  0.336700  0.333186     numpy_arrays/array-2.npy
3        3  0.866066  0.260158     numpy_arrays/array-3.npy
4        4  0.519107  0.106170     numpy_arrays/array-4.npy
...    ...       ...       ...                          ...
9995  9995  0.919494  0.640358  numpy_arrays/array-9995.npy
9996  9996  0.911520  0.739093  numpy_arrays/array-9996.npy
9997  9997  0.026405  0.342753  numpy_arrays/array-9997.npy
9998  9998  0.789790  0.531555  numpy_arrays/array-9998.npy
9999  9999  0.505488  0.449314  numpy_arrays/array-9999.npy

[10000 rows x 4 columns]
Numpy arrays are described in numpy_arrays.csv


The script created a csv-file, where we have a running index (`n`), some hyperparameters (`param1`, `param2`) and the name of the numpy file that is connected to these hyperparameters. It also created 10000 100x100 random numpy arrays and stored them in numpy_arrays.

In [3]:
import pandas as pd

pd.read_csv('numpy_arrays.csv').head()

,n,param1,param2,npyfile
0,0,0.215664,0.185119,numpy_arrays/array-0.npy
1,1,0.038707,0.180713,numpy_arrays/array-1.npy
2,2,0.336700,0.333186,numpy_arrays/array-2.npy
3,3,0.866066,0.260158,numpy_arrays/array-3.npy
4,4,0.519107,0.106170,numpy_arrays/array-4.npy


## Creating a parquet-file (from command line)

Now we can create our parquet-file, where the data in the numpy-arrays is stored in the parquet file with the hyperparameters.

In [4]:
!python -m npy2parquet numpy_arrays.csv numpy_arrays.parquet --npyfile-column npyfile --data-column data --batch-size 20 --overwrite

Creating dataset "numpy_arrays.parquet" based on "numpy_arrays.csv"
File numpy_arrays.parquet exists, removing it.
Splitting the dataframe with 10000 rows to 500 batches of 20.


This creates a parquet file that contains all of the data in the `.npy`-files. Parquet-format stores data in batches, which makes it possible to load files that are larger than our RAM. It also helps with IO.

If we want, we can randomize the order of the data before creation. If one needs to access the data in random order (such as in machine learning), it is a good idea to randomize the data order before writing the parquet-file.

In [5]:
!python -m npy2parquet numpy_arrays.csv numpy_arrays.parquet --npyfile-column npyfile --data-column data --batch-size 20 --shuffle --seed 10 --overwrite

Creating dataset "numpy_arrays.parquet" based on "numpy_arrays.csv"
File numpy_arrays.parquet exists, removing it.
Shuffling labels with seed 10.
Splitting the dataframe with 10000 rows to 500 batches of 20.


Now we can compare the size of the parquet-file to the size of the numpy arrays:

In [6]:
!du -sh numpy_arrays

782M	numpy_arrays


In [7]:
!du -sh numpy_arrays.parquet

1,2G	numpy_arrays.parquet


## Creating a parquet-file (from Python)

We can also create the parquet-file from Python.

In [8]:
import pandas
import npy2parquet

numpy_df = pd.read_csv('numpy_arrays.csv')

npy2parquet.df2parquet(
    numpy_df,
    'numpy_arrays.parquet',
    npyfile_column='npyfile',
    data_column='data',
    batch_size=20,
    overwrite=True)

File numpy_arrays.parquet exists, removing it.
Splitting the dataframe with 10000 rows to 500 batches of 20.


## Iterating over the parquet-file (sequential order)

One way of accessing the parquet-file is through sequential order. We can use the `iter_parquet`-function to iterate over the parquet.

What the function does is it loads one batch at a time from the parquet-file, loads it into memory, and returns the data one row at a time.

In [9]:
from npy2parquet import iter_parquet
for df, data in iter_parquet('numpy_arrays.parquet', data_column='data'):
    print(f'First returned  object is of type: {type(df)}\n')
    print(f'Values of the first object:\n{df}\n')
    print(f'Second returned object is of type: {type(data)}\n')
    print(f'Shape of the second object:\n{data.shape}\n')
    break

First returned  object is of type: <class 'pandas.core.frame.DataFrame'>

Values of the first object:
   n    param1    param2                   npyfile
0  0  0.215664  0.185119  numpy_arrays/array-0.npy

Second returned object is of type: <class 'numpy.ndarray'>

Shape of the second object:
(100, 100)



We can compare these values to our initial data:

In [10]:
numpy_df = pd.read_csv('numpy_arrays.csv')

print(f'First row of our dataframe:\n{numpy_df.head(1)}\n')

numpy_array_1 = np.load(numpy_df.loc[0,'npyfile'])

print(f'Shape of our first numpy array:\n{numpy_array_1.shape}\n')

print(f'Data in our numpy arrays match:\n{np.all(numpy_array_1 == data)}\n')

First row of our dataframe:
   n    param1    param2                   npyfile
0  0  0.215664  0.185119  numpy_arrays/array-0.npy

Shape of our first numpy array:
(100, 100)

Data in our numpy arrays match:
True



## Iterating over parquet-file (random-order)

For random order iteration, we should first create our parquet-file with shuffle enabled.

In [11]:
import pandas
import npy2parquet

numpy_df = pd.read_csv('numpy_arrays.csv')

npy2parquet.df2parquet(
    numpy_df,
    'numpy_arrays.parquet',
    npyfile_column='npyfile',
    data_column='data',
    batch_size=100,
    shuffle=True,
    seed=10,
    overwrite=True)

File numpy_arrays.parquet exists, removing it.
Shuffling labels with seed 10.
Splitting the dataframe with 10000 rows to 100 batches of 100.


Now we can use iter_parquet with shuffling. This is not a complete shuffle, but a pseudorandom shuffle. Basically, the order of the batches will be randomized and the data within a shuffle will be returned in a random order. However, one rarely needs a full random shuffle.

In [12]:
for df, data in iter_parquet('numpy_arrays.parquet', data_column='data', shuffle=True, seed=10):
    print(f'Returned random row:\n{df}')
    break

Shuffling labels with seed 10.
Returned random row:
         n    param1    param2                      npyfile
6500  7965  0.593723  0.374821  numpy_arrays/array-7965.npy


Changing the seed will change the batch ordering and ordering within the batch, but it will not recreate the batches, thus it is important to give the dataset a full shuffle while it's being created.

## Iterating over parquet-file (maximum performance)

One can also return named tuples from the iteration, if they are easier to use while iterating (see [itertuples](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.itertuples.html#pandas.DataFrame.itertuples) for more information).

This will dramatically increase the iteration speed as the parameters do not need to be converted into DataFrames.

In [13]:
for param_tuple, data in iter_parquet('numpy_arrays.parquet', data_column='data', return_tuples=True):
    print(f'Returned random row as a named tuple:\n{param_tuple}')
    print(f'Value of param1: {param_tuple.param1}')
    break

Returned random row as a named tuple:
Pandas(Index=0, n=5937, param1=0.5332069718172633, param2=0.8945746473961547, npyfile='numpy_arrays/array-5937.npy')
Value of param1: 0.5332069718172633


## Verifying data integrity

Storing data in a parquet-file is good, but it is important to make certain that the data is ordered correctly and hyperparameters match the data. The function `verify_parquet` will iterate over the parquet-dataset and it will compare each row of hyperparameters to a given `DataFrame`. In addition, it will compare the data stored in the parquet-file to data stored in the numpy file.

Warning: this will read through the whole data.

In [14]:
import pandas as pd
from npy2parquet import verify_parquet

numpy_arrays_df = pd.read_csv('numpy_arrays.csv')

verify_parquet('numpy_arrays.parquet', numpy_arrays_df, npyfile_column='npyfile', data_column='data')

Ran tests for numpy_arrays.parquet:

DataFrame tests:
Number of rows:                     10000
Number of matching rows:            0
Number of mismatching rows:         10000

Numpy arrays tests:
Number of arrays:                   10000
Number of matching numpy arrays:    10000
Number of mismatching numpy arrays: 0


(False, True)

## Performance comparison

The biggest benefit of using parquet-files comes from the reduced IO. `read_npy_test.py` is a small code that calculates a mean of all of the arrays we have created.

Let's use `strace` to measure the number of IO calls created by this operation.

In [15]:
!time strace -c -e trace=%file,read,write python read_npy_test.py

Average mean of 1000 arrays: 0.500112
Average mean of 2000 arrays: 0.500070
Average mean of 3000 arrays: 0.500084
Average mean of 4000 arrays: 0.500044
Average mean of 5000 arrays: 0.500022
Average mean of 6000 arrays: 0.500018
Average mean of 7000 arrays: 0.500031
Average mean of 8000 arrays: 0.500041
Average mean of 9000 arrays: 0.500048
Average mean of 10000 arrays: 0.500045
Average mean of arrays: 0.500045
% time     seconds  usecs/call     calls    errors syscall
------ ----------- ----------- --------- --------- ----------------
 79,41    0,069900           1     50374           read
 19,17    0,016874           1     10273        37 openat
  1,30    0,001148           1       774        90 stat
  0,05    0,000048           6         7           getcwd
  0,04    0,000031           5         6         2 readlink
  0,02    0,000019           1        11           write
  0,00    0,000000           0         1           lstat
  0,00    0,000000           0         1         1 access

With parquet, the amount of operations is reduced by a significant factor. This is because parquet loads the data one batch at a time.

In [16]:
!time strace -c -e trace=%file,read,write python read_parquet_test.py

Average mean of 1000 arrays: 0.499930
Average mean of 2000 arrays: 0.500001
Average mean of 3000 arrays: 0.500040
Average mean of 4000 arrays: 0.500031
Average mean of 5000 arrays: 0.500019
Average mean of 6000 arrays: 0.500020
Average mean of 7000 arrays: 0.500015
Average mean of 8000 arrays: 0.500020
Average mean of 9000 arrays: 0.500021
Average mean of 10000 arrays: 0.500045
Average mean of arrays: 0.500045
% time     seconds  usecs/call     calls    errors syscall
------ ----------- ----------- --------- --------- ----------------
 43,41    0,002146           0      2216       276 stat
 35,69    0,001764           1      1421           read
 19,93    0,000985           1       785        80 openat
  0,73    0,000036           3        11           write
  0,18    0,000009           1         7           getcwd
  0,06    0,000003           0         7         3 readlink
  0,00    0,000000           0         1           open
  0,00    0,000000           0         1           lstat
 